In [1]:
import hashlib
import random
import math

In [2]:
q = 2^23 - 2^13 +1
print (q)
n = 256
k = 3
l = 2
eta = 7

Zx.<x> = ZZ[]
R.<x> = Zx.quotient(x^n + 1)
print(R)

F = GF(q)
Rq = PolynomialRing(F, name="x")
print(Rq)
x = (Rq).gen()
X = x

g = (x^n + 1)
xi = g.roots(multiplicities=False)[-1]
rs = [xi^(2*i+1)  for i in range(n)] 
base = crt_basis([(x - r) for r in rs])

8380417
Univariate Quotient Polynomial Ring in x over Integer Ring with modulus x^256 + 1
Univariate Polynomial Ring in x over Finite Field of size 8380417


In [3]:
def is_even(number):
    if mod(number,2) == 0:
        return True

def nicelift(u,alfa):
    return lift(u + (alfa/2)) - (alfa/2)

# mod^+-
def centered_reduction_modulo_q(alfa,r):
    try:
        assert(alfa >= 0)
    except:
        print('alfa isn\'t positive -> FAILURE')
        return False
    
    new_r = mod(r,alfa)
    if is_odd(alfa):
        #assert nicelift(r,alfa)
        assert ( new_r >= -(alfa/2)) and (new_r <= (alfa/2))
    else:
        #new_r = nicelift(r,alfa-1)
        assert (new_r >= -(alfa/2)) and (new_r <= (alfa/2))
    return new_r

# mod^+
def reduction_modulo_q(alfa,r):
    try:
        assert(alfa >= 0)
    except:
        print('alfa isn\'t positive -> FAILURE')
        return False
    new_r = mod(r,alfa)
    assert (new_r >=0 and new_r < alfa)
    return new_r


In [4]:
def checkSize_S_eta(w,etra):
    if w in Rq:
        w_inf = abs(centered_reduction_modulo_q(w,q))
        return w_inf
    else:
        assert w in R
        my_max = -1000
        total = 0
        for i in range(0,len(w)):
            w_i_inf = abs(centered_reduction_modulo_q(w,q))
            if (w_i_inf < my_max):
                my_max = w_i_inf
            total += (w_i_inf)**2
        '''
        JAVARDICES
        '''
        
    
    

## Supporting Algorithms

In [5]:
def power_2_round_q(r,d):
    r = mod(r,q)
    r0 = mod(2**d,r)
    return ((r-r0)/2**d,r0)

def Decompose_q(r,alfa):
    r = mod(r,q)
    r0 = mod(r,alfa)
    if (r- r0 == q-1):
        r1 = 0
        r0 = r0 -1
    else:
        r1 = (r1-r0)/alfa
    return (r1,r0)

def HighBits_q(r,alfa):
    (r1,r0) = Decompose_q(r,alfa)
    return r1

def LowBits_q(r,alfa):
    (r1,r0) = Decompose_q(r,alfa)
    return r0

def MakeHint_q(z,r,alfa):
    r1 = HighBits_q(r,alfa)
    v1 = HighBits_q(r+z, alfa)
    return r1 != v1

def UseHint_q(h,r,alfa):
    m = (q-1)/alfa
    (r1,r0) = Decompose_q(r,alfa)
    if(h == 1 and r0>0):
        return mod(r1+1,m)
    if(h == 1 and r0<=0):
        return mod(r1-1,m)
    return r1

In [6]:
def SampleInBall():
    c = []
    for j in range(0,256):
        c.append(0)
    for i in range(196,256):
        j = randint(0,i)
        s = randint(0,1)
        c[i] = c[j]
        c[j]=(-1)**s
    return c

def count_0s_1s(c):
    total_0 = 0
    total_1= 0
    for i in range(0,len(c)):
        if c[i] == 0:
            total_0 += 1
        else:
            total_1 += 1
    return (total_0,total_1)

In [7]:
c = SampleInBall()
total0,total1 = count_0s_1s(c)

print('0\'s :' + str(total0))
print('1\'s :' + str(total1))

0's :196
1's :60


In [8]:
'''
def random_pol(args=None):
    if args in None:
        return Rq.random_element(1024)
    return Rq.random_element(args)
'''

'\ndef random_pol(args=None):\n    if args in None:\n        return Rq.random_element(1024)\n    return Rq.random_element(args)\n'

## NTT

In [9]:
'''
Number-theoretic transform
'''
def ntt(f):
    def _expand_(f): 
        u = f.list()
        return u + [0]*(n-len(u)) 

    def _ntt_(xi,N,f):
        if N==1:
            return f
        N_ = N/2 ; xi2 =  xi^2  
        f0 = [f[2*i]   for i in range(N_)] ; f1 = [f[2*i+1] for i in range(N_)] 
        ff0 = _ntt_(xi2,N_,f0) ; ff1 = _ntt_(xi2,N_,f1)  

        s  = xi ; ff = [F(0) for i in range(N)] 
        for i in range(N_):
            a = ff0[i] ; b = s*ff1[i]  
            ff[i] = a + b ; ff[i + N_] = a - b 
            s = s * xi2                     
        return ff 

    return _ntt_(xi,n,_expand_(f))

def ntt_inv(ff):                           
    return sum([ff[i]*base[i] for i in range(n)])

def random_pol(args=None):
    return (Rq).random_element(args)

def Poly_mult( c1, c2):
    r = [None] * n
    for i in range(0, n):
        r[i] =  (c1[i]*c2[i] % q)
    return r

## Conversions

In [10]:
'''
Recebe um polinómio e transforma-o numa lista de coeficientes
'''
def Poly_to_List(pol):
    ret = []
    for coef in pol:
        ret.append(coef)
    return ret
'''
Recebe uma lista de polinómios e transforma-a numa lista de listas de coeficientes
'''
def PolyList_to_Coefs_Lists(poly_List):
    ret = []
    for poly in poly_List:
        ret.append(Poly_to_List(poly))
    return ret
    
    '''
    ret = []
    for poly in poly_List:
        aux = []
        for coef in poly:
            aux.append(coef)
        ret.append(aux)
    return ret
    '''
'''
Recebe uma lista de coeficientes e transforma-a num polinómio
'''
def List_to_Poly(my_list):
    return Zx(my_list) 

'''
Recebe uma lista de listas de coeficientes e transforma-a numa lista de polinómios
'''
def List_Lists_to_Poly_Lists(my_list):
    ret = []
    for coefs in my_list:
        ret.append(List_to_Poly(coefs))
    return ret
'''
Recebe um objeto de bytes e transforma-o numa lista de bytes com **size** cada.
'''
def bytes_to_bytesList(my_byte_string,size):
    new_Array = []
    for i in range(0,len(my_byte_string),size):
        new_Array.append(my_byte_string[i:i+size])
    return new_Array
'''
Recebe uma lista de bytes e transforma-a numa lista de listas de inteiros.
'''
def bytesList_to_int_ListsOfList(bytesList):
    new_lists_list = []
    for nbytes in bytesList:
        aux = []
        for nb in nbytes:
            aux.append(nb)
        new_lists_list.append(aux)
    return new_lists_list

'''
Recebe uma lista de listas de inteiros e transforma-a numa matriz **lines** X **columns** de polinómios
'''
def int_Lists_to_matrix_polys(my_list,lines,columns):
    assert len(my_list) == lines*columns
    ret = []
    index = 0
    for i in range(0,lines):
        aux= []
        for j in range(0,columns):
            aux.append(Zx(my_list[index]))
            index += 1
        ret.append(aux)
    return ret
'''
Recebe uma lista de polinómios e transforma-a numa matriz **lines**  X **columns**
'''
def polys_List_to_Matrix(my_list,lines,columns):
    return Matrix(lines,columns,my_list)

'''
Recebe uma matriz e transforma-a numa lista de polinómios
'''
def Matrix_to_coefs_List(my_matrix):
    ret = []
    for line in my_matrix:
        for column in line:
            ret.append(Poly_to_List(column))
    return ret

In [35]:
def Poly_mult( c1, c2):
    r = [None] * len(c1)
    for i in range(0, len(c1)):
        r[i] =  (c1[i]*c2[i] % q)
    return r

def Poly_sum(c1,c2):
    r = [None] * len(c1)
    for i in range(0,len(c1)):
        r[i] = c1[i] + c2[i]
    return r

def Matrix_Line_Mult(lineA,columnB):
    my_sum = 0
    ret = [None] * len(lineA)
    for index in range(0,len(lineA)):
        ret[index] = Poly_mult(lineA[index],columnB[index])
    my_sum = Poly_sum(ret[0],ret[1])
    for index in range(1,len(ret)-1):
        my_sum = Poly_sum(my_sum,ret[index+1])
    return my_sum 

def A_Matrix_Mult_S1(A,s1):
    assert len(A[0]) == len(s1)
    print(' \n ... Dimensões corretas ... \n ')
    ret = []
    for line in A:
        ret.append(Matrix_Line_Mult(line,s1))
    return ret
            
    
    

## Dilithium algorithm

In [36]:
import hashlib
def SHAKE(X,L,S = None):
    if S is None:
        return hashlib.shake_256(b'0x01'+X).digest(int(L))
'''
def ExpandA2(seed):
    new_seed = SHAKE(seed,k*l)
    #print(new_seed)
    lines = []
    coefs = []
    index = 0
    for i in range(0,k):
        aux= []
        for j in range(0,l):
            aux.append(Rq(new_seed[index]))
            coefs.append(Rq(new_seed[index]))
            index += 1
        lines.append(aux)
    #print(lines)
    A = matrix(Rq,k,l,lines)
    print(A[0])
    #my_ntt = NTT(n)
    inv_A = ntt(A)
'''

'''
Recebo uma seed de 256 bits.
Através do SHAKE transformo-a numa seed de k*l*256 bits. (no nosso caso 3*2*256).

Pego nessa nova seed e transformo-a numa lista de seeds de 256 bits (de tamanho 6).
Cada seed desta nova lista transformo-a numa lista de inteiros em Rq.

Agora a cada sublista aplicamos uma transformação para Rq e criamos a matriz através desses valores

Por último aplicamos um ntt à matriz A e devolvemos essa tranformação.

'''    
    
def ExpandA(seed):
    new_seed = SHAKE(seed,k*l*len(seed))
    
    new_seed_Array = bytes_to_bytesList(new_seed,len(seed))
    #print(new_seed_Array)
    
    new_seeds_list = bytesList_to_int_ListsOfList(new_seed_Array)
    #print(new_seeds_list)
    
    poly_seeds_list = int_Lists_to_matrix_polys(new_seeds_list,k,l)
    #print(poly_seeds_list[0])
    
    #print(len(poly_seeds_list[0]))
    A = polys_List_to_Matrix(poly_seeds_list,k,l)
    #print(A)
    
    #ntt_A = ntt(A)
    #print((ntt_A[0]))    
    #coefs_ntt_A = PolyList_to_Coefs_Lists(ntt_A)
    #return coefs_ntt_A

    return A
'''
def generateS(): 
    s1_bytes = []
    for i in range(0,l):
        s1_bytes.append(os.urandom(eta))
    #print(s1_bytes)
    s1_array= []
    for s1_b in s1_bytes:
        aux = []
        for b in s1_b:
            aux.append(b)
        s1_array.append(aux)
    print(s1_array)
    s1 = []
    for position in s1_array:
        s1.append(Rq((position)))
    print(s1)
    
    #s2 = 0
    assert len(s1) == l
    #print('1')
    
    s2_bytes = []
    for i in range(0,k):
        s2_bytes.append(os.urandom(eta))
    print(s2_bytes)
    s2_array= []
    for s2_b in s2_bytes:
        aux = []
        for b in s2_b:
            aux.append(b)
        s2_array.append(aux)
    #print(s2_array)
    s2 = []
    for position in s2_array:
        s2.append(Rq(Zx(position)))
            
    assert len(s2) == k

    return (Matrix(s1),Matrix(s2))
'''

def generateS1(l):
    seed = bytearray(os.urandom(eta)) #criamos uma seed para s1
    new_seed = SHAKE(seed,l*(256)) #aumentamos para tamanho l*256
    
    seed_bytesList = bytes_to_bytesList(new_seed,256) #uma lista de l elementos de tamanho 256
    
    seed_intLists = bytesList_to_int_ListsOfList(seed_bytesList) #lista de listas de coeficientes
    
    return int_Lists_to_matrix_polys(seed_intLists,l,1) #coluna com l polinómios
    


In [37]:
def Gen():
    ro = bytearray(os.urandom(256))
    #print(ro)
    print('Temos **rô**.')    
    
    K = bytearray(os.urandom(256))
    print('Temos **K**.')
    
    #(s1,s2)  = generateS()
    #print(s1)
    
    A = ExpandA(ro)
    print('Temos a matriz **A**.') #Lista de listas de coeficientes da matriz A[k][l]
    #print(len(A[0]))
    
    s1 = generateS1(l)
    #print(s1)
    
    t = A_Matrix_Mult_S1(A,s1)
    #t = (A,s1)
    
Gen()    

Temos **rô**.
Temos **K**.
Temos a matriz **A**.
 
 ... Dimensões corretas ... 
 


TypeError: object of type 'sage.rings.polynomial.polynomial_integer_dense_flint.Polynomial_integer_dense_flint' has no len()

SyntaxError: invalid syntax (<ipython-input-32-228f566371d3>, line 1)